玩家在艾泽拉斯漫游时长遇到天气变化
搭建和开启魔兽世界模拟器服务端和mysql数据库
navicat新建查询:
use world;
select * from game_weather into outfile 'C:\Users\*\zone.xlsx';
将mysql查询结果导出为zone.xls表,将数据结构复制到nodpad+修改格式后赋值给names.

In [1]:
import pandas as pd
names = ['zone','spring_rain_chance','spring_snow_chance','spring_storm_chance','summer_rain_chance','summer_snow_chance','summer_storm_chance','fall_rain_chance','fall_snow_chance','fall_storm_chance','winter_rain_chance','winter_snow_chance','winter_storm_chance','ScriptName']
zone = pd.read_csv('zone.xlsx', sep = '\t', index_col = None, names = names, engine='python')
zone.head()

,zone,spring_rain_chance,spring_snow_chance,spring_storm_chance,summer_rain_chance,summer_snow_chance,summer_storm_chance,fall_rain_chance,fall_snow_chance,fall_storm_chance,winter_rain_chance,winter_snow_chance,winter_storm_chance,ScriptName
0,1,0,25,0,0,15,0,0,25,0,0,25,0,NaN
1,3,0,0,20,0,0,20,0,0,20,0,0,15,NaN
2,10,15,0,0,15,0,0,20,0,0,15,0,0,NaN
3,11,25,0,0,15,0,0,25,0,0,25,0,0,NaN
4,12,20,0,0,15,0,0,20,0,0,20,0,0,NaN


表内含有控制客户端天气状况的参数和区域代码(zone),这是类似华东,中原,江南,西北的描述,并非具体地区名.
遍历world库,并没有地区名与zone匹配,要寻找的匹配信息想必被封装在客户端内.
一个思路是用gm命令直接在客户端内匹配出地区,将结果保存到map.xlsx
mysql查询:
SELECT name,help
FROM `command`
where name like '%go%';
得到gm命令:Syntax: .go zonexy #x #y [#zone]

In [2]:
'''读取这个map.xlsx,然后与zone合并'''
map_df = pd.read_excel('map.xlsx', sheet_name='Sheet1')
map_df.tail()

,zone,area
30,3358,NaN
31,3428,安其拉神殿
32,3429,安其拉废墟
33,3521,扭曲虚空
34,4080,北海


In [3]:
concat_df = map_df.merge(zone, on = ['zone'])
concat_df.tail()

,zone,area,spring_rain_chance,spring_snow_chance,spring_storm_chance,summer_rain_chance,summer_snow_chance,summer_storm_chance,fall_rain_chance,fall_snow_chance,fall_storm_chance,winter_rain_chance,winter_snow_chance,winter_storm_chance,ScriptName
30,3358,NaN,10,0,0,10,0,0,10,0,0,10,0,0,NaN
31,3428,安其拉神殿,0,0,20,0,0,20,0,0,20,0,0,20,NaN
32,3429,安其拉废墟,0,0,20,0,0,20,0,0,20,0,0,20,NaN
33,3521,扭曲虚空,10,0,0,15,0,0,20,0,0,10,0,0,NaN
34,4080,北海,20,0,0,20,0,0,20,0,0,10,0,0,NaN


清洗zone列,ScriptName列,和area列中的NaN

In [4]:
del concat_df['ScriptName']
del concat_df['zone']
df = concat_df.T
df.dropna(axis = 1,inplace = True)
df

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,31,32,33,34
area,无尽之海,洛克莫丹,埃尔文森林,湿地,暴风城,莫高雷,提瑞斯法林地,西部荒野,银松森林,丹莫罗,...,希利苏斯,黑海岸,血色修道院,菲拉斯,祖尔格拉布,斯坦索姆,安其拉神殿,安其拉废墟,扭曲虚空,北海
spring_rain_chance,0,0,15,25,20,25,10,20,0,15,...,15,0,5,0,15,5,0,0,10,20
spring_snow_chance,25,0,0,0,0,0,0,0,20,0,...,0,25,0,0,0,0,0,0,0,0
spring_storm_chance,0,20,0,0,0,0,0,0,0,0,...,0,0,0,20,0,0,20,20,0,0
summer_rain_chance,0,0,15,15,15,20,15,25,0,15,...,10,0,10,0,5,5,0,0,15,20
summer_snow_chance,15,0,0,0,0,0,0,0,20,0,...,0,20,0,0,0,0,0,0,0,0
summer_storm_chance,0,20,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,20,20,0,0
fall_rain_chance,0,0,20,25,20,25,15,25,0,15,...,20,0,25,0,15,5,0,0,20,20
fall_snow_chance,25,0,0,0,0,0,0,0,25,0,...,0,20,0,0,0,0,0,0,0,0
fall_storm_chance,0,20,0,0,0,0,0,0,0,0,...,0,0,0,20,0,0,20,20,0,0


In [5]:
'''设置area为列索引'''
df.columns = [df.loc['area']]
df.drop(index='area',inplace = True)
df

area,无尽之海,洛克莫丹,埃尔文森林,湿地,暴风城,莫高雷,提瑞斯法林地,西部荒野,银松森林,丹莫罗,...,希利苏斯,黑海岸,血色修道院,菲拉斯,祖尔格拉布,斯坦索姆,安其拉神殿,安其拉废墟,扭曲虚空,北海
spring_rain_chance,0,0,15,25,20,25,10,20,0,15,...,15,0,5,0,15,5,0,0,10,20
spring_snow_chance,25,0,0,0,0,0,0,0,20,0,...,0,25,0,0,0,0,0,0,0,0
spring_storm_chance,0,20,0,0,0,0,0,0,0,0,...,0,0,0,20,0,0,20,20,0,0
summer_rain_chance,0,0,15,15,15,20,15,25,0,15,...,10,0,10,0,5,5,0,0,15,20
summer_snow_chance,15,0,0,0,0,0,0,0,20,0,...,0,20,0,0,0,0,0,0,0,0
summer_storm_chance,0,20,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,20,20,0,0
fall_rain_chance,0,0,20,25,20,25,15,25,0,15,...,20,0,25,0,15,5,0,0,20,20
fall_snow_chance,25,0,0,0,0,0,0,0,25,0,...,0,20,0,0,0,0,0,0,0,0
fall_storm_chance,0,20,0,0,0,0,0,0,0,0,...,0,0,0,20,0,0,20,20,0,0
winter_rain_chance,0,0,15,25,20,25,10,20,0,15,...,15,0,5,0,15,5,0,0,10,10


不妨依据此表查看暴风城在当前季节下的天气状况

In [6]:
df['暴风城'].loc[['winter_rain_chance', 'winter_snow_chance', 'winter_storm_chance']]

area,暴风城
winter_rain_chance,20
winter_snow_chance,0
winter_storm_chance,0


看来艾泽拉斯的暴风城在冬季只会下雨,不会下雪.接下来暴风城会下雨的p值为20,不过winter_rain_chance这个参数未必是百分比